## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [3]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt
import IPython.display as ipd

import sys
sys.path.append('waveglow/')
import numpy as np
import torch
device = torch.device('cpu')
from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import CustomText
from denoiser import Denoiser

ct = CustomText(symbol_name='phoneme')

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "C:\Users\cuong.nm164980\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')
    fig.savefig('demo.png', bbox_inches='tight')
    plt.show()


#### Setup hparams

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050
print(hparams)

#### Load model from checkpoint

In [ ]:
checkpoint_path = "F:\\learn\DoAnTotNghiep\\checkpoint\\Tacotron2_phoneme_7000"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cpu().eval()

#### Load WaveGlow for mel2audio synthesis and denoiser

In [ ]:
waveglow_path = 'F:\\learn\\DoAnTotNghiep\\checkpoint\\Waveglow_checkpoint_78000'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow).cuda()

#### Prepare text input

In [ ]:
text = "{x i_T0 n ch a_T1 o_T0 t aa_T2 t k a_T3 k a_T2 k b a_T5 n} ."
sequence = np.array(ct.text_to_sequence(text, []))[None, :]
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cpu().long()

#### Decode text input and plot results

In [12]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))


C:\Users\cuong.nm164980\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  import sys
C:\Users\cuong.nm164980\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  if __name__ == '__main__':


In [8]:
plt.show()

C:\Users\cuong.nm164980\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  """Entry point for launching an IPython kernel.


#### Synthesize audio from spectrogram using WaveGlow

In [9]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet.cuda().half(), sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [10]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 